In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from platform import python_version
print(f"Python version: {python_version()}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

In [ ]:
pd.set_option('display.max_columns', None)

# Carga de datos
El archivo de datos proveído del historial de la Facultad de Ingeniería es *datosConcatenados.csv*. Se procede a manipularlo utilizando `DataFrame` de pandas para tener una mejor organización y acceso.

In [ ]:
df = pd.read_csv("data/datosConcatenados.csv")
df.head()